In [6]:
# connect to test.db sqlite, create session, then run "get all users" with session.Query
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
from models import Base, User, Location, ExternalAddress, Cast, EthTransaction
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

engine = create_engine(os.getenv("PLANETSCALE_URL"))
# if you have the link you can do
# engine = create_engine('mysql+pymysql://...')

In [8]:
# get all users with bio that contains "coinbase"

with sessionmaker(bind=engine)() as session:
    # get transaction with highest block number
    tx = session.query(EthTransaction).order_by(EthTransaction.block_num.desc()).first()
    # coinbase_users = session.query(User).filter(
    #     User.bio_text.contains("coinbase")).all()
    
    # for user in coinbase_users:
    #     print(user.username)
    print(tx.block_num)

16839706
